Reducer Function

In [48]:
import os
import json
from typing_extensions import TypedDict, Annotated
from typing import List, Dict
from dotenv import load_dotenv

from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from operator import add
from langgraph.graph import END, START, StateGraph

class State(TypedDict):
    messages: Annotated[list, add]
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field

load_dotenv()
gemini_api_key = os.getenv("GEMINI")


class IELTSESSAY(TypedDict):
    """Represents the state of the essay evaluation workflow."""
    essay: str
    language_feedback: str
    clarity_feedback: str
    analyse_feedback: str
    overall_feedback: str
    individual_score: Annotated[List[int], add]
    average_score: str


    
class Evaluate_responce(BaseModel):
    """
    The structured output format for each evaluation.
    This ensures we get feedback and a score in a predictable format.
    """
    feedback: str = Field(..., description="Write detailed feedback in 50 words about the essay's specific aspect.")
    score: int = Field(..., description="Grant a numerical score from 1 to 10.")


def evaluate_language(state : IELTSESSAY)->IELTSESSAY:
    essay = state["essay"]
    prompt = "you are require to anayse th easy and rate the language and score it between 1 and 10 the essay is this {essay}"
    responce = llm_struct_output.invoke(prompt)
    return {"language_feedback":responce.feedback,"individual_score":[responce.score]}
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=gemini_api_key)

llm_struct_output = llm.with_structured_output(Evaluate_responce)





In [49]:
def evaluate_language(state: IELTSESSAY) -> IELTSESSAY:
    """Evaluates the essay's language and vocabulary."""
    essay_text = state['essay']
    prompt = f"Evaluate the language usage, vocabulary, grammar, and syntax of the following IELTS essay. Provide detailed feedback and a score from 1 to 10. Essay:\n\n{essay_text}"
    response = llm_struct_output.invoke(prompt)
    
    return {
        'language_feedback': response.feedback,
        'individual_score': [response.score]
    }

def evaluate_clarity(state: IELTSESSAY) -> IELTSESSAY:
    """Evaluates the essay's clarity and coherence."""
    essay_text = state['essay']
    prompt = f"Evaluate the clarity, coherence, and paragraph structure of the following IELTS essay. Provide detailed feedback and a score from 1 to 10. Essay:\n\n{essay_text}"
    response = llm_struct_output.invoke(prompt)
    
    return {
        'clarity_feedback': response.feedback,
        'individual_score': [response.score]
    }

def evaluate_analysis(state: IELTSESSAY) -> IELTSESSAY:
    """Evaluates the essay's analytical quality and argument strength."""
    essay_text = state['essay']
    prompt = f"Evaluate the logical flow, depth of analysis, and strength of arguments in the following IELTS essay. Provide detailed feedback and a score from 1 to 10. Essay:\n\n{essay_text}"
    response = llm_struct_output.invoke(prompt)
    
    return {
        'analyse_feedback': response.feedback,
        'individual_score': [response.score]
    }

def aggregate_scores(state: IELTSESSAY) -> IELTSESSAY:
    """Calculates the average score from all individual scores."""
    scores = state['individual_score']
    print("scores : ", scores)
    if scores:
        average = sum(scores) / len(scores)
        return {
            'average_score': f"{average:.2f}"
        }
    return {'average_score': "N/A"}

def generate_overall_feedback(state: IELTSESSAY) -> IELTSESSAY:
    """Generates a final, overall summary of the essay's strengths and weaknesses."""
    essay_text = state['essay']
    prompt = f"Provide a concise, overall summary of the strengths and weaknesses of this IELTS essay. Do not give a score. Essay:\n\n{essay_text}"
    response = llm.invoke(prompt).content
    
    return {
        'overall_feedback': response
    }

In [ ]:
graph = StateGraph(IELTSESSAY)


graph.add_node("language", evaluate_language)
graph.add_node("clarity", evaluate_clarity)
graph.add_node("analysis", evaluate_analysis)
graph.add_node("overall", generate_overall_feedback)

graph.add_node("aggregate", aggregate_scores)


graph.add_edge(START, "language")
graph.add_edge(START, "clarity")
graph.add_edge(START, "analysis")
graph.add_edge(START, "overall")

graph.add_edge("language", "aggregate")
graph.add_edge("clarity", "aggregate")
graph.add_edge("analysis", "aggregate")
graph.add_edge("overall", "aggregate")


graph.add_edge("aggregate", END)

workflow = graph.compile()


sample_essay = """
There is a sense of complexity today that has led many to believe the individual investor has little chance of competing with professional brokers and investment firms. However, Malkiel states this is a major misconception as he explains in his book “A Random Walk Down Wall Street”. What does a random walk mean? The random walk means in terms of the stock market that, “short term changes in stock prices cannot be predicted”. So how does a rational investor determine which stocks to purchase to maximize returns? Chapter 1 begins by defining and determining the difference in investing and speculating. Investing defined by Malkiel is the method of “purchasing assets to gain profit in the form of reasonably predictable income or appreciation over the long term”. Speculating in a sense is predicting, but without sufficient data to support any kind of conclusion. What is investing? Investing in its simplest form is the expectation to receive greater value in the future than you have today by saving income rather than spending. For example a savings account will earn a particular interest rate as will a corporate bond. Investment returns therefore depend on the allocation of funds and future events. Traditionally there have been two approaches used by the investment community to determine asset valuation: “the firm-foundation theory” and the “castle in the air theory”. The firm foundation theory argues that each investment instrument has something called intrinsic value, which can be determined analyzing securities present conditions and future growth. The basis of this theory is to buy securities when they are temporarily undervalued and sell them when they are temporarily overvalued in comparison to there intrinsic value One of the main variables used in this theory is dividend income. A stocks intrinsic value is said to be “equal to the present value of all its future dividends”. This is done using a method called discounting. Another variable to consider is the growth rate of the dividends. The greater the growth rate the more valuable the stock. However it is difficult to determine how long growth rates will last. Other factors are risk and interest rates, which will be discussed later. Warren Buffet, the great investor of our time, used this technique in making his fortune."""

initial_state = {
    "essay": sample_essay, 
    "language_feedback": "",
    "clarity_feedback": "",
    "analyse_feedback": "",
    "overall_feedback": "",
    "individual_score": [],
    "average_score": "",
}


final_state = workflow.invoke(initial_state)

print(json.dumps(final_state, indent=2))


{
  "essay": "\nThere is a sense of complexity today that has led many to believe the individual investor has little chance of competing with professional brokers and investment firms. However, Malkiel states this is a major misconception as he explains in his book \u201cA Random Walk Down Wall Street\u201d. What does a random walk mean? The random walk means in terms of the stock market that, \u201cshort term changes in stock prices cannot be predicted\u201d. So how does a rational investor determine which stocks to purchase to maximize returns? Chapter 1 begins by defining and determining the difference in investing and speculating. Investing defined by Malkiel is the method of \u201cpurchasing assets to gain profit in the form of reasonably predictable income or appreciation over the long term\u201d. Speculating in a sense is predicting, but without sufficient data to support any kind of conclusion. What is investing? Investing in its simplest form is the expectation to receive grea